# Parameter Tunning

We start with the parameter tunning. We have done some earl training and we got a good accuracy. 55% on the training set.

The code done before is the following one.

    model.add(Conv2D(64, (3, 3), padding = "same", activation = "relu", input_shape = (128, 217, 1)))
    model.add(Conv2D(64, (3, 3), padding = "same", activation = "relu"))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (3, 3), padding = "same", activation = "relu"))
    model.add(Conv2D(128, (3, 3), padding = "same", activation = "relu"))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.1))
    model.add(Dense(1024, activation = "relu"))
    model.add(Dense(50, activation = "softmax"))
    
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

In [1]:
import librosa
import librosa.display

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import re

In [2]:
def create_melspec(params, audio_data, sampling_rate):
    S = librosa.feature.melspectrogram(audio_data, sr=sampling_rate, n_mels=params['n_mels'],
                                      hop_length=params['hop_length'], n_fft=params['n_fft'],
                                      fmin=params['fmin'], fmax=(sampling_rate//2))
    S_dB = librosa.power_to_db(S, ref=np.max)
    S_dB = S_dB.astype(np.float32)

    return S

In [3]:
def load_audio(single_file):
    # Take in count that our dataset files lasts everyone of them the same time. If not you should do something.
    audio, sr = librosa.load(single_file, duration=5.0)
    return audio, sr

In [4]:
    melspec_params = {
        'n_mels': 128, # The entire frequency spectrum divided by a concrete number.
        'duration': 5*22050, # Number of seconds * sample rate
        'hop_length': 512, # It has something to do with the duration. I think it fills the space with repetitions
        'n_fft': 2048, # Length of the Fast Fourier Transformation
        'fmin': 20
}

In [5]:
files_list = os.listdir('../Datasets/s_a_d__datasets/audio_esc50')
paths_list = []
for file in files_list:
    paths_list.append('../Datasets/s_a_d__datasets/audio_esc50/'+file)
    
files_list_csv = os.listdir('../Datasets/s_a_d__datasets/experimental')

In [6]:
Images = []
Labels = []

for file in files_list_csv:
    image_to_be = pd.read_csv('../Datasets/s_a_d__datasets/experimental/'+file)
    Images.append(image_to_be.to_numpy())
    Labels.append(re.split(r'-|\.', file)[3])

In [7]:
Images = np.asarray(Images)
Labels = np.asarray(Labels)

Labels = pd.get_dummies(Labels)

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from keras.callbacks import EarlyStopping
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import Adam

In [3]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(Images, Labels, test_size=0.3, stratify=Labels)

In [11]:
X_train = X_train.reshape(1400, 128, 217, 1)
X_test = X_test.reshape(600, 128, 217, 1)


input_dim = (128,217,1)

In [5]:
model = Sequential()

### Adding some Dense layers.

First adding only one.

In [6]:
model.add(Conv2D(64, (3, 3), padding = "same", activation = "relu", input_shape = (128, 217, 1)))
model.add(Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.1))
model.add(Dense(1024, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(512, activation="relu"))
model.add(Dense(50, activation = "softmax"))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 217, 64)      640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 217, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 108, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 108, 128)      73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 108, 128)      147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 54, 128)       0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 128)               0

In [14]:
model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics = ['accuracy'])

callbacks = [EarlyStopping(monitor='accuracy', patience=5, verbose=1, mode='auto')]

In [15]:
model.fit(X_train, y_train, epochs=50, batch_size=36, callbacks=callbacks)

Epoch 1/50
 2/39 [>.............................] - ETA: 1:29 - loss: 4.4307 - accuracy: 0.0139

KeyboardInterrupt: 

In [51]:
predictions = model.predict(X_test)
score = model.evaluate(X_test, y_test)
print(score)

19/19 [==============================] - 5s 256ms/step - loss: 2.3551 - accuracy: 0.3833
[2.355092763900757, 0.38333332538604736]


In [15]:
model.add(Conv2D(32, (3, 3), padding = "same", activation = "relu", input_shape = (128, 217, 1)))
model.add(Conv2D(32, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.1))
model.add(Dense(1024, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(512, activation="relu"))
model.add(Dense(50, activation = "softmax"))

In [18]:
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics = ['accuracy'])

callbacks = [EarlyStopping(monitor='accuracy', patience=5, verbose=1, mode='auto')]

In [19]:
model.fit(X_train, y_train, epochs=100, batch_size=36, callbacks=callbacks)

Epoch 1/100
39/39 [==============================] - 82s 2s/step - loss: 3.8902 - accuracy: 0.0193
Epoch 2/100
39/39 [==============================] - 84s 2s/step - loss: 3.7885 - accuracy: 0.0379
Epoch 3/100
39/39 [==============================] - 82s 2s/step - loss: 3.7350 - accuracy: 0.0436
Epoch 4/100
39/39 [==============================] - 85s 2s/step - loss: 3.6824 - accuracy: 0.0443
Epoch 5/100
39/39 [==============================] - 98s 3s/step - loss: 3.5775 - accuracy: 0.0564
Epoch 6/100
39/39 [==============================] - 97s 2s/step - loss: 3.4569 - accuracy: 0.0636
Epoch 7/100
39/39 [==============================] - 105s 3s/step - loss: 3.2496 - accuracy: 0.0914
Epoch 8/100
39/39 [==============================] - 102s 3s/step - loss: 3.1249 - accuracy: 0.1064
Epoch 9/100
39/39 [==============================] - 98s 3s/step - loss: 3.0417 - accuracy: 0.1314
Epoch 10/100
39/39 [==============================] - 96s 2s/step - loss: 2.9293 - accuracy: 0.1486
Epoch 1

In [20]:
predictions = model.predict(X_test)
score = model.evaluate(X_test, y_test)
print(score)

19/19 [==============================] - 5s 267ms/step - loss: 2.5200 - accuracy: 0.5517
[2.5199925899505615, 0.5516666769981384]


In [17]:
model = Sequential()

In [18]:
model.add(Conv2D(32, (3, 3), padding = "same", activation = "relu", input_shape = (128, 217, 1)))
model.add(Conv2D(32, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.1))
model.add(Dense(1024, activation = "relu"))
model.add(Dense(50, activation = "softmax"))

In [19]:
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics = ['accuracy'])

callbacks = [EarlyStopping(monitor='accuracy', patience=5, verbose=1, mode='auto')]

In [20]:
model.fit(X_train, y_train, epochs=100, batch_size=30, callbacks=callbacks)

Epoch 1/100
47/47 [==============================] - 90s 2s/step - loss: 3.8743 - accuracy: 0.0264
Epoch 2/100
47/47 [==============================] - 92s 2s/step - loss: 3.7767 - accuracy: 0.0457
Epoch 3/100
47/47 [==============================] - 92s 2s/step - loss: 3.6531 - accuracy: 0.0629
Epoch 4/100
47/47 [==============================] - 90s 2s/step - loss: 3.4641 - accuracy: 0.0729
Epoch 5/100
47/47 [==============================] - 90s 2s/step - loss: 3.2892 - accuracy: 0.0964
Epoch 6/100
47/47 [==============================] - 104s 2s/step - loss: 3.1692 - accuracy: 0.1271
Epoch 7/100
47/47 [==============================] - 102s 2s/step - loss: 3.0578 - accuracy: 0.1521
Epoch 8/100
47/47 [==============================] - 102s 2s/step - loss: 2.9473 - accuracy: 0.1714
Epoch 9/100
47/47 [==============================] - 96s 2s/step - loss: 2.8012 - accuracy: 0.1936
Epoch 10/100
47/47 [==============================] - 93s 2s/step - loss: 2.6810 - accuracy: 0.2193
Epoch 

In [21]:
predictions = model.predict(X_test)
score = model.evaluate(X_test, y_test)
print(score)

19/19 [==============================] - 8s 412ms/step - loss: 2.7727 - accuracy: 0.5850
[2.772711753845215, 0.5849999785423279]


In [13]:
model = Sequential()

In [14]:
model.add(Conv2D(16, (3, 3), padding = "same", activation = "relu", input_shape = (128, 217, 1)))
model.add(Conv2D(16, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), padding = "same", activation = "relu"))
model.add(Conv2D(32, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.1))
model.add(Dense(1024, activation = "relu"))
model.add(Dense(50, activation = "softmax"))

In [15]:
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics = ['accuracy'])

callbacks = [EarlyStopping(monitor='accuracy', patience=5, verbose=1, mode='auto')]

In [34]:
model.fit(X_train, y_train, epochs=100, batch_size=36, callbacks=callbacks)

Epoch 1/100
39/39 [==============================] - 41s 1s/step - loss: 3.8580 - accuracy: 0.0271
Epoch 2/100
39/39 [==============================] - 42s 1s/step - loss: 3.7861 - accuracy: 0.0557
Epoch 3/100
39/39 [==============================] - 42s 1s/step - loss: 3.6701 - accuracy: 0.0679
Epoch 4/100
39/39 [==============================] - 47s 1s/step - loss: 3.5443 - accuracy: 0.0657
Epoch 5/100
39/39 [==============================] - 46s 1s/step - loss: 3.4086 - accuracy: 0.0750
Epoch 6/100
39/39 [==============================] - 46s 1s/step - loss: 3.1762 - accuracy: 0.1100
Epoch 7/100
39/39 [==============================] - 46s 1s/step - loss: 2.9973 - accuracy: 0.1357
Epoch 8/100
39/39 [==============================] - 46s 1s/step - loss: 2.9717 - accuracy: 0.1629
Epoch 9/100
39/39 [==============================] - 46s 1s/step - loss: 2.7513 - accuracy: 0.1979
Epoch 10/100
39/39 [==============================] - 46s 1s/step - loss: 2.6071 - accuracy: 0.2271
Epoch 11/

In [35]:
predictions = model.predict(X_test)
score = model.evaluate(X_test, y_test)
print(score)

19/19 [==============================] - 4s 220ms/step - loss: 2.1331 - accuracy: 0.6317
[2.1330630779266357, 0.6316666603088379]


### Trying the tensorboard.

In [16]:
root_logdir = os.path.join(os.curdir, 'my_logs')

In [17]:
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

In [18]:
run_logdir = get_run_logdir()

In [19]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit(X_train, y_train, epochs=30,
                    batch_size=30, callbacks=[tensorboard_cb])

Epoch 1/30
 1/47 [..............................] - ETA: 0s - loss: 3.9403 - accuracy: 0.0000e+00WARNING:tensorflow:From /home/toni_domenech/Python/envs/s_a_d/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
47/47 [==============================] - 41s 877ms/step - loss: 3.8514 - accuracy: 0.0221
Epoch 2/30
47/47 [==============================] - 44s 945ms/step - loss: 3.7449 - accuracy: 0.0464
Epoch 3/30
47/47 [==============================] - 42s 897ms/step - loss: 3.6247 - accuracy: 0.0550
Epoch 4/30
47/47 [==============================] - 42s 887ms/step - loss: 3.4486 - accuracy: 0.0707
Epoch 5/30
47/47 [==============================] - 46s 974ms/step - loss: 3.2242 - accuracy: 0.1086
Epoch 6/30
47/47 [==============================] - 46s 970ms/step - loss: 3.0855 - accuracy: 0.1336
Epo

### New model viewing the results of HParam tunning.

In [20]:
%load_ext tensorboard

In [15]:
from datetime import datetime

In [17]:
# Create a TensorBoard callback
logs = "another_logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')

In [18]:
callbacks = [EarlyStopping(monitor='accuracy', patience=5, verbose=1, mode='auto')]

In [23]:
model = Sequential()

In [24]:
model.add(Conv2D(64, (3, 3), padding = "same", activation = "relu", input_shape = (128, 217, 1)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(256, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(512, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.1))
model.add(Dense(1024, activation = "relu"))
model.add(Dense(50, activation = "softmax"))

In [25]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
 model.fit(X_train, y_train, epochs=100, batch_size=30, callbacks=[tboard_callback, callbacks])

Epoch 1/100
47/47 [==============================] - 87s 2s/step - loss: 5.2662 - accuracy: 0.0279
Epoch 2/100
47/47 [==============================] - 91s 2s/step - loss: 3.7079 - accuracy: 0.0721
Epoch 3/100
47/47 [==============================] - 92s 2s/step - loss: 3.5170 - accuracy: 0.0850
Epoch 4/100
47/47 [==============================] - 92s 2s/step - loss: 3.1792 - accuracy: 0.1350
Epoch 5/100
47/47 [==============================] - 92s 2s/step - loss: 3.0464 - accuracy: 0.1643
Epoch 6/100
47/47 [==============================] - 92s 2s/step - loss: 2.8298 - accuracy: 0.2229
Epoch 7/100
47/47 [==============================] - 92s 2s/step - loss: 2.6586 - accuracy: 0.2593
Epoch 8/100
47/47 [==============================] - 92s 2s/step - loss: 2.4417 - accuracy: 0.3114
Epoch 9/100
47/47 [==============================] - 91s 2s/step - loss: 2.2831 - accuracy: 0.3550
Epoch 10/100
47/47 [==============================] - 95s 2s/step - loss: 2.1270 - accuracy: 0.4050
Epoch 11/

In [27]:
predictions = model.predict(X_test)
score = model.evaluate(X_test, y_test)
print(score)

19/19 [==============================] - 11s 566ms/step - loss: 5.9653 - accuracy: 0.6067
[5.965286731719971, 0.6066666841506958]


In [28]:
%tensorboard --logdir logs